# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Context.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Context

This notebook contains a `Context` object to manage Solana connection configuration and Mango groups.

## Environment Variables

It's possible to override the values in the `Context` variables provided. This can be easier than creating the `Context` in code or introducing dependencies and configuration.

The following environment variables are read:
* CLUSTER (defaults to: mainnet-beta)
* CLUSTER_URL (defaults to URL for RPC server for CLUSTER defined in `ids.json`)
* GROUP_NAME (defaults to: BTC_ETH_USDT)


## Provided Configured Objects

This notebook provides 3 `Context` objects, already configured and ready to use.
* default_context (uses the environment variables specified above and `ids.json` file for configuration)
* solana_context (uses the environment variables specified above and `ids.json` file for configuration but explicitly sets the RPC server to be [Solana's mainnet RPC server](https://api.mainnet-beta.solana.com))
* serum_context (uses the environment variables specified above and `ids.json` file for configuration but explicitly sets the RPC server to be [Project Serum's mainnet RPC server](https://solana-api.projectserum.com))

Where notebooks depend on `default_context`, you can change this behaviour by adding an import line like:
```
from Context import solana_context as default_context
```
This can be useful if one of the RPC servers starts behaving oddly.

In [ ]:
import logging
import os
import time
import typing

from decimal import Decimal
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.rpc.types import MemcmpOpts, TokenAccountOpts, RPCMethod, RPCResponse
from solana.rpc.commitment import Single
from spl.token._layouts import ACCOUNT_LAYOUT

from Constants import MangoConstants, SOL_DECIMAL_DIVISOR
from Decoder import decode_binary, encode_binary


## AccountInfo class

This - unlike the classes in [BaseModel](BaseModel.ipynb) - is quite low-level and needs to be handled differently. It's here because then the `Context` object can depend on it, while putting it in [BaseModel](BaseModel.ipynb) would introduce a dependency loop.

In [ ]:
class AccountInfo:
    def __init__(self, address: PublicKey, executable: bool, lamports: Decimal, owner: PublicKey, rent_epoch: Decimal, data: bytes):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.address: PublicKey = address
        self.executable: bool = executable
        self.lamports: Decimal = lamports
        self.owner: PublicKey = owner
        self.rent_epoch: Decimal = rent_epoch
        self.data: bytes = data

    def encoded_data(self) -> typing.List:
        return encode_binary(self.data)

    def __str__(self) -> str:
        return f"""« AccountInfo [{self.address}]:
    Owner: {self.owner}
    Executable: {self.executable}
    Lamports: {self.lamports}
    Rent Epoch: {self.rent_epoch}
»"""

    def __repr__(self) -> str:
        return f"{self}"

    @staticmethod
    def _from_response_values(response_values: typing.Dict[str, typing.Any], address: PublicKey) -> "AccountInfo":
        executable = bool(response_values["executable"])
        lamports = Decimal(response_values["lamports"])
        owner = PublicKey(response_values["owner"])
        rent_epoch = Decimal(response_values["rentEpoch"])
        data = decode_binary(response_values["data"])
        return AccountInfo(address, executable, lamports, owner, rent_epoch, data)

    @staticmethod
    def from_response(response: RPCResponse, address: PublicKey) -> "AccountInfo":
        return AccountInfo._from_response_values(response["result"]["value"], address)


## Context class

In [ ]:
class Context:
    def __init__(self, cluster: str, cluster_url: str, program_id: PublicKey, dex_program_id: PublicKey,
                 group_name: str, group_id: PublicKey):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.cluster: str = cluster
        self.cluster_url: str = cluster_url
        self.client: Client = Client(cluster_url)
        self.program_id: PublicKey = program_id
        self.dex_program_id: PublicKey = dex_program_id
        self.group_name: str = group_name
        self.group_id: PublicKey = group_id
        self.commitment = Single
        self.encoding:str = "base64"


    def fetch_sol_balance(self, account_public_key: PublicKey) -> Decimal:
        result = self.client.get_balance(account_public_key, commitment=self.commitment)
        value = Decimal(result["result"]["value"])
        return value / SOL_DECIMAL_DIVISOR


    def fetch_token_balance(self, account_public_key: PublicKey, token_mint: PublicKey) -> typing.Optional[Decimal]:
        opts = TokenAccountOpts(mint = token_mint)

        token_accounts_response = self.client.get_token_accounts_by_owner(account_public_key, opts, commitment=self.commitment)
        token_accounts = token_accounts_response["result"]["value"]
        if len(token_accounts) == 0:
            return None

        total_value = Decimal(0)
        for token_account in token_accounts:
            result = self.client.get_token_account_balance(token_account["pubkey"], commitment=self.commitment)
            value = Decimal(result["result"]["value"]["amount"])
            decimal_places = result["result"]["value"]["decimals"]
            divisor = Decimal(10 ** decimal_places)
            total_value += value / divisor

        return total_value


    def fetch_largest_token_account_for_owner(self, owner_public_key: PublicKey, token_mint: PublicKey) -> typing.Optional[AccountInfo]:
        opts = TokenAccountOpts(mint = token_mint)

        token_accounts_response = self.client.get_token_accounts_by_owner(owner_public_key, opts, commitment=self.commitment)

        largest_amount = -1
        largest_account: typing.Optional[AccountInfo] = None
        for token_account_response in token_accounts_response["result"]["value"]:
            data = decode_binary(token_account_response["account"]["data"])
            account_data = ACCOUNT_LAYOUT.parse(data)
            if account_data.amount > largest_amount:
                largest_amount = account_data.amount
                largest_account = AccountInfo(PublicKey(token_account_response["pubkey"]),
                                              token_account_response["account"]["executable"],
                                              Decimal(token_account_response["account"]["lamports"]),
                                              PublicKey(token_account_response["account"]["owner"]),
                                              Decimal(token_account_response["account"]["rentEpoch"]),
                                              data)

        return largest_account


    def load_account(self, address: PublicKey) -> typing.Optional["AccountInfo"]:
        response: RPCResponse = self.client.get_account_info(address)
        if "error" in response:
            message = response["error"]["message"]
            code = response["error"]["code"]
            raise Exception(f"Error loading account - error message from server: '{message}', code: {code}")

        if response["result"]["value"] is None:
            return None

        return AccountInfo.from_response(response, address)


    def load_multiple_accounts(self, addresses: typing.List[PublicKey]) -> typing.List["AccountInfo"]:
        address_strings = list(map(PublicKey.__str__, addresses))
        response = self.client._provider.make_request(RPCMethod("getMultipleAccounts"), address_strings)
        response_value_list = zip(response["result"]["value"], addresses)
        return list(map(lambda pair: AccountInfo._from_response_values(pair[0], pair[1]), response_value_list))


    def fetch_program_accounts_for_owner(self, program_id: PublicKey, owner: PublicKey):
        memcmp_opts = [
            MemcmpOpts(offset=40, bytes=str(owner)),
        ]

        return self.client.get_program_accounts(program_id, memcmp_opts=memcmp_opts, commitment=self.commitment, encoding=self.encoding)


    @staticmethod
    def _lookup_name_by_address(address: PublicKey, collection: typing.Dict[str, str]) -> typing.Optional[str]:
        address_string = str(address)
        for stored_name, stored_address in collection.items():
            if address_string == stored_address:
                return stored_name
        return None

    def lookup_market_name(self, market_address: PublicKey) -> str:
        return Context._lookup_name_by_address(market_address, MangoConstants[self.cluster]["spot_markets"]) or "« Unknown Market »"

    def lookup_oracle_name(self, token_address: PublicKey) -> str:
        return Context._lookup_name_by_address(token_address, MangoConstants[self.cluster]["oracles"]) or "« Unknown Oracle »"

    def lookup_token_name(self, token_address: PublicKey) -> str:
        return Context._lookup_name_by_address(token_address, MangoConstants[self.cluster]["symbols"]) or "« Unknown Token »"

    def wait_for_confirmation(self, transaction_id: str, max_wait_in_seconds: int = 60) -> None:
        for wait in range(0, max_wait_in_seconds):
            time.sleep(1)
            confirmed = default_context.client.get_confirmed_transaction(transaction_id)
            if confirmed["result"] is not None:
                print(f"Confirmed after {wait} seconds.")
                return
        print(f"Timed out after {wait} seconds waiting on transaction {transaction_id}.")

    def __str__(self) -> str:
        return f"""« Context:
    Cluster: {self.cluster}
    Cluster URL: {self.cluster_url}
    Program ID: {self.program_id}
    DEX Program ID: {self.dex_program_id}
    Group Name: {self.group_name}
    Group ID: {self.group_id}
»"""

    def __repr__(self) -> str:
        return f"{self}"


## default_context object

A default `Context` object that connects to mainnet, to save having to create one all over the place. This `Context` uses the default values in the `ids.json` file, overridden by environment variables if they're set.

In [ ]:
default_cluster = os.environ.get("CLUSTER") or "mainnet-beta"
default_cluster_url = os.environ.get("CLUSTER_URL") or MangoConstants["cluster_urls"][default_cluster]

default_program_id = PublicKey(MangoConstants[default_cluster]["mango_program_id"])
default_dex_program_id = PublicKey(MangoConstants[default_cluster]["dex_program_id"])

default_group_name = os.environ.get("GROUP_NAME") or "BTC_ETH_USDT"
default_group_id = PublicKey(MangoConstants[default_cluster]["mango_groups"][default_group_name]["mango_group_pk"])

default_context = Context(default_cluster, default_cluster_url, default_program_id,
                          default_dex_program_id, default_group_name, default_group_id)

## solana_context object

A `Context` object that connects to mainnet using Solana's own https://api.mainnet-beta.solana.com server. Apart from the RPC server URL, this `Context` uses the default values in the `ids.json` file, overridden by environment variables if they're set.

In [ ]:
solana_cluster_url = "https://api.mainnet-beta.solana.com"

solana_context = Context(default_cluster, solana_cluster_url, default_program_id,
                         default_dex_program_id, default_group_name, default_group_id)

## serum_context object

A `Context` object that connects to mainnet using Serum's own https://solana-api.projectserum.com server. Apart from the RPC server URL, this `Context` uses the default values in the `ids.json` file, overridden by environment variables if they're set.

In [ ]:
serum_cluster_url = "https://solana-api.projectserum.com"

serum_context = Context(default_cluster, serum_cluster_url, default_program_id,
                        default_dex_program_id, default_group_name, default_group_id)

# 🏃 Running

If running interactively, just print out the default Context object.

In [ ]:
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)

    print(default_context)

    single_account_info = default_context.load_account(default_context.dex_program_id)
    print("DEX account info", single_account_info)

    multiple_account_info = default_context.load_multiple_accounts([default_context.program_id, default_context.dex_program_id])
    print("Mango program and DEX account info", multiple_account_info)

    print("Lookup ETH token name result:", default_context.lookup_token_name(PublicKey("2FPyTwcZLUg1MDrwsyoP4D6s1tM7hAkHYRjkNb5w6Pxk")))
    print("Lookup BTC/USDC market name result:", default_context.lookup_market_name(PublicKey("CVfYa8RGXnuDBeGmniCcdkBwoLqVxh92xB1JqgRQx3F")))

    # Fill out your account address between the quotes below
    MY_ACCOUNT_ADDRESS = ""
    # Don't edit anything beyond here.

    if MY_ACCOUNT_ADDRESS != "":
        account_key = PublicKey(MY_ACCOUNT_ADDRESS)
        print("SOL balance:", default_context.fetch_sol_balance(account_key))
        print("SRM balance:", default_context.fetch_token_balance(account_key, PublicKey("SRMuApVNdxXokk5GT7XD5cUUgXMBCoAz2LHeuAoKWRt")))
        print("SRM AccountInfo:", default_context.fetch_largest_token_account_for_owner(account_key, PublicKey("SRMuApVNdxXokk5GT7XD5cUUgXMBCoAz2LHeuAoKWRt")))
